In [4]:
import sys
sys.path.append("/tf/ProjectGabriel/pilca")
import matplotlib.pyplot as plt
import os
import json
import datetime
import numpy as np
import torch
import pandas as pd

import utils.torchphysics as tp
import utils.utils as ut 
import torch
from lightcurve_fitting import filters as flc


torch.set_default_dtype(torch.float64)


In [27]:
BASE_DIR = os.path.join(ut.storage_parent_path, "experiments")
EXPERIMENTS = [os.path.join(ut.storage_parent_path, "experiments", dir) for dir in os.listdir(BASE_DIR)]
EXPERIMENTS_DIR = EXPERIMENTS[0]


In [28]:
os.listdir(EXPERIMENTS_DIR)

['exp_',
 'exp__filters-z_time-2d',
 'summary_all.csv',
 'exp__filters-z_time-1d',
 'exp__filters-z-y_time-2d',
 'exp__filters-z-y_time-1d']

In [ ]:

# ==============================================
# 🧩 1. Collect all experiments
# ==============================================
records = []

for exp in sorted([f for f in os.listdir(EXPERIMENTS_DIR) if f.startswith("exp_f")]):
    exp_path = os.path.join(EXPERIMENTS_DIR, exp)

    if not os.path.isdir(exp_path):
        continue

    config_path = os.path.join(exp_path, "exp_config.json")
    loss_csv_path = os.path.join(exp_path, "loss_history.csv")
    summary_path = os.path.join(exp_path, "summary.txt")

    if not os.path.exists(config_path):
        continue

    with open(config_path, "r") as f:
        cfg = json.load(f)

    # Load last loss value (if available)
    if os.path.exists(loss_csv_path):
        loss_df = pd.DataFrame({"loss", np.load(loss_csv_path.replace("csv", npy))})#pd.read_csv(loss_csv_path)
        final_loss = loss_df["loss"].iloc[-1]
    else:
        final_loss = np.nan

    summary = ""
    if os.path.exists(summary_path):
        with open(summary_path, "r") as f:
            summary = f.read().strip()

    records.append({
        "exp_name": exp,
        "filters": "-".join(cfg.get("filters", [])),
        "time_span": cfg.get("time_span"),
        "final_loss": final_loss,
        "n_points": cfg.get("n_points", None),
        "hidden_dim": cfg.get("hidden_dim"),
        "lr": cfg.get("learning_rate"),
        "epochs": cfg.get("epochs"),
        "summary": summary
    })


In [23]:
records



[{'exp_name': 'exp_',
  'filters': 'z-y',
  'time_span': 2.0,
  'final_loss': nan,
  'n_points': 60.0,
  'hidden_dim': None,
  'lr': None,
  'epochs': None,
  'summary': "Filters=['z', 'y'], Time=2d, Final loss=0.0375"},
 {'exp_name': 'exp__filters-z_time-1d',
  'filters': 'z',
  'time_span': 1.0,
  'final_loss': nan,
  'n_points': 14.0,
  'hidden_dim': None,
  'lr': None,
  'epochs': None,
  'summary': "Filters=['z'], Time=1d, Final loss=0.0220"},
 {'exp_name': 'exp__filters-z_time-2d',
  'filters': 'z',
  'time_span': 2.0,
  'final_loss': nan,
  'n_points': 29.0,
  'hidden_dim': None,
  'lr': None,
  'epochs': None,
  'summary': "Filters=['z'], Time=2d, Final loss=0.0458"}]

In [25]:

# ==============================================
# 📊 2. Create summary dataframe
# ==============================================
df = pd.DataFrame(records)
df = df.sort_values(by=["filters", "time_span"]).reset_index(drop=True)
print(f"Loaded {len(df)} experiments.")
display(df.head())

# Save master summary CSV
summary_csv = os.path.join(EXPERIMENTS_DIR, "summary_all.csv")
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")


Loaded 3 experiments.


,exp_name,filters,time_span,final_loss,n_points,hidden_dim,lr,epochs,summary
0,exp__filters-z_time-1d,z,1.0,NaN,14.0,None,None,None,"Filters=['z'], Time=1d, Final loss=0.0220"
1,exp__filters-z_time-2d,z,2.0,NaN,29.0,None,None,None,"Filters=['z'], Time=2d, Final loss=0.0458"
2,exp_,z-y,2.0,NaN,60.0,None,None,None,"Filters=['z', 'y'], Time=2d, Final loss=0.0375"


Summary saved to: /tf/astrodados2/pilca.storage/experiments/TEST_0/summary_all.csv


In [26]:

# ==============================================
# 📈 3. Visualize loss evolution per experiment
# ==============================================
def plot_loss(exp_name):
    exp_path = os.path.join(EXPERIMENTS_DIR, exp_name)
    loss_csv_path = os.path.join(exp_path, "loss_history.csv")
    if not os.path.exists(loss_csv_path):
        print("No loss history for:", exp_name)
        return
    df_loss = pd.read_csv(loss_csv_path)
    plt.figure(figsize=(6,4))
    plt.plot(df_loss["epoch"], df_loss["loss"], label=exp_name)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Loss Curve: {exp_name}")
    plt.grid(True)
    plt.legend()
    plt.show()
# Example: plot the last experiment's loss
plot_loss(df.iloc[-1]["exp_name"])

No loss history for: exp_


In [ ]:



# ==============================================
# 🌈 4. Compare final losses across filters/time
# ==============================================
plt.figure(figsize=(10,5))
for filt in df["filters"].unique():
    sub = df[df["filters"] == filt]
    plt.plot(sub["time_span"], sub["final_loss"], marker="o", label=filt)
plt.xlabel("Time span (days)")
plt.ylabel("Final loss")
plt.title("Final loss vs time span per filter combination")
plt.grid(True)
plt.legend(title="Filters")
plt.show()

# ==============================================
# 📦 5. (Optional) Inspect posterior samples
# ==============================================
example_exp = df.iloc[-1]["exp_name"]
samples_path = os.path.join(EXPERIMENTS_DIR, example_exp, "samples.npy")
if os.path.exists(samples_path):
    samples = np.load(samples_path)
    plt.figure(figsize=(10,4))
    plt.hist(samples[:,0], bins=50, alpha=0.7, label="param 0")
    plt.hist(samples[:,1], bins=50, alpha=0.7, label="param 1")
    plt.legend()
    plt.title(f"Posterior samples ({example_exp})")
    plt.show()
else:
    print("No samples.npy found in", example_exp)